In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model


In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Correct path for the uploaded dataset file
dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'

# Load the dataset
df = pd.read_csv(dataset_path)

# Display the first few rows of the dataframe to confirm it's loaded correctly
#print(df.head())


   Unnamed: 0                                              Image  \
0        3452  ['https://images-na.ssl-images-amazon.com/imag...   
1        1205  ['https://images-na.ssl-images-amazon.com/imag...   
2        1708  ['https://images-na.ssl-images-amazon.com/imag...   
3        2078  ['https://images-na.ssl-images-amazon.com/imag...   
4         801  ['https://images-na.ssl-images-amazon.com/imag...   

                                         Review Text  
0  Loving these vintage springs on my vintage str...  
1  Works great as a guitar bench mat. Not rugged ...  
2  We use these for everything from our acoustic ...  
3  Great price and good quality.  It didn't quite...  
4  I bought this bass to split time as my primary...  


In [8]:
import torch
import os
from torchvision import models, transforms
import requests
from PIL import Image, UnidentifiedImageError
from io import BytesIO
import pandas as pd
import pickle
import numpy as np
import nltk

# Ensure NLTK resources are available
nltk.download('stopwords')
nltk.download('wordnet')

output_dir = '/content/drive/My Drive/IR_Assignment2'

# Define image preprocessing pipeline
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the pre-trained ResNet model
resnet = models.resnet50(pretrained=True)
resnet.eval()  # Set model to evaluation mode

def extract_image_features(url):
    """Extract image features from a given URL using ResNet50."""
    try:
        if url.startswith("[") and url.endswith("]"):
            # Convert string list representation to actual list
            url = eval(url)[0]

        response = requests.get(url)
        img = Image.open(BytesIO(response.content)).convert('RGB')  # Convert to RGB
        img_t = image_transforms(img)
        img_t = img_t.unsqueeze(0)  # Add batch dimension
        with torch.no_grad():  # No need for gradient computation
            features = resnet(img_t)
        return features.cpu().numpy().flatten()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
    except UnidentifiedImageError:
        print(f"Cannot identify image file from URL {url}.")
    except Exception as e:
        print(f"Unexpected error for URL {url}: {e}")
    return None

# Load dataset
dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'  # Update this path
df = pd.read_csv(dataset_path)

extracted_features = []

for _, row in df.iterrows():
    feature_vector = extract_image_features(row['Image'])  # Ensure 'Image' is the correct column name
    if feature_vector is not None:
        extracted_features.append(feature_vector)

# Save the results
features_file_path = os.path.join(output_dir, 'image_features_refactored.pkl')
with open(features_file_path, 'wb') as f:
    pickle.dump(extracted_features, f)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/71F3npeHUDL._SY88.jpg.
Cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/71B8OOE5N8L._SY88.jpg.
Cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/718niQ1GEwL._SY88.jpg.
Cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/61OboZT-kcL._SY88.jpg.
Cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/710a2Pyh5lL._SY88.jpg.
Cannot identify image file from URL https://images-na.ssl-images-amazon.com/images/I/816NMd0LexL._SY88.jpg.


In [9]:
import os
import re
import math
import pickle
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Download necessary NLTK datasets
nltk.download(['punkt', 'stopwords', 'wordnet'])

def text_normalization_and_processing(input_text):
    """Normalize and process text data for analysis."""
    # Normalize text to lowercase
    normalized_text = input_text.lower()
    # Remove URLs and user mentions with regular expressions
    cleaned_text = re.sub(r'\b(http\S+|@\w+)\b', '', normalized_text)
    # Tokenize while removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(cleaned_text)
    # Filter out stopwords and apply lemmatization
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in english_stopwords]
    return filtered_tokens

# Set up environment for text processing
english_stopwords = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

# Load dataset
dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'  # Update this path
df = pd.read_csv(dataset_path)
# Preprocess reviews
reviews = df['Review Text'].fillna(' ').apply(str).tolist()
processed_reviews = [text_normalization_and_processing(review) for review in reviews]

def tf_idf_computation(docs_tokens):
    """Compute the TF-IDF for each document."""
    # Term Frequency calculation
    tf = [Counter(doc_tokens) for doc_tokens in docs_tokens]
    # Document Frequency and Inverse Document Frequency
    doc_count = float(len(docs_tokens))
    idf = {}
    for doc in tf:
        for term in doc.keys():
            idf[term] = idf.get(term, 0) + 1
    for term, val in idf.items():
        idf[term] = math.log(doc_count / val)

    # TF-IDF calculation
    tf_idf = []
    for doc in tf:
        doc_tf_idf = {}
        for term, val in doc.items():
            doc_tf_idf[term] = val * idf[term]
        tf_idf.append(doc_tf_idf)

    return tf_idf

tf_idf_scores_refined = tf_idf_computation(processed_reviews)

# Prepare directory for saving processed data
data_dir = '/content/drive/My Drive/IR_Assignment2'
os.makedirs(data_dir, exist_ok=True)

# Saving processed data
pickle.dump(processed_reviews, open(os.path.join(data_dir, 'refined_reviews.pkl'), 'wb'))
pickle.dump(tf_idf_scores_refined, open(os.path.join(data_dir, 'refined_tf_idf.pkl'), 'wb'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
import numpy as np
import pandas as pd
import pickle
import requests
from PIL import Image
from io import BytesIO
import torch
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import re
from collections import Counter

# Setting up file paths
data_csv_path = '/content/drive/My Drive/IR_Assignment2/Data_A2.csv'
output_path = '/content/drive/My Drive/IR_Assignment2'
img_feat_path = os.path.join(output_path, 'img_feats.pkl')
text_feat_path = os.path.join(output_path, 'text_features.pkl')

# Loading the dataset and features
data_frame = pd.read_csv(data_csv_path)
with open(img_feat_path, 'rb') as img_file:
    img_feats = pickle.load(img_file)
with open(text_feat_path, 'rb') as text_file:
    text_features = pickle.load(text_file)

# Pretrained model for image feature extraction
model = models.resnet50(pretrained=True)
model.eval()

# Transformation pipeline for images
img_transform = Compose([
    Resize(256),
    CenterCrop(224),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Calculate cosine similarity
def calc_cosine_sim(vector1, vectors):
    norm_vector1 = vector1 / np.linalg.norm(vector1)
    norm_vectors = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
    return np.dot(norm_vectors, norm_vector1)

# Identify most similar items
def get_top_matches(processed, features, num_matches=3):
    sim_scores = calc_cosine_sim(processed, np.array(features))
    top_idxs = np.argsort(sim_scores)[-num_matches:][::-1]
    return top_idxs, sim_scores[top_idxs]

# Text processing and TF calculation
def process_text(text):
    cleaned_text = re.sub(r'\W+', ' ', text.lower())
    words = cleaned_text.split()
    return Counter(words)

def calc_tf(word_counts):
    total_count = sum(word_counts.values())
    return {word: count / total_count for word, count in word_counts.items()}

# Image processing and feature extraction
def process_img(url):
    try:
        img_response = requests.get(url)
        img = Image.open(BytesIO(img_response.content)).convert('RGB')
        img_processed = img_transform(img).unsqueeze(0)
        with torch.no_grad():
            img_features = model(img_processed)
        return img_features.numpy().flatten()
    except Exception as error:
        print(f"Failed to process image: {error}")
        return None

# Input handling
img_url = input("Image URL: ")
review_text = input("Review Text: ")

# Processing inputs
token_counts = process_text(review_text)
tf_review = calc_tf(token_counts)
processed_img = process_img(img_url)

if processed_img is not None:
    # Finding similar images and reviews
    img_indices, img_sims = get_top_matches(processed_img, list(img_feats.values()))
    review_indices, review_sims = get_top_matches(tf_review, list(text_features.values()))

    print(f"Similar Image Indices: {img_indices}\nImage Similarities: {img_sims}")
    print(f"Similar Review Indices: {review_indices}\nReview Similarities: {review_sims}")
else:
    print("Failed to find the requested image or review in the dataset.")

# Storing results
results = {
    'image_indices': img_indices,
    'image_similarities': img_sims,
    'review_indices': review_indices,
    'review_similarities': review_sims,
}
with open(os.path.join(output_path, 'results_refactored.pkl'), 'wb') as result_file:
    pickle.dump(results, result_file)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Enter the image URL:  https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg
Enter the review text: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
Similar Image Indices: [  0  62 193]
Image Similarities: [0.8524184, 0.836084, 0.8211998]
Similar Review Indices: [  0 794 750]
Review Similarities: [0.14284236057674865, 0.14002800840280094, 0.12369368723729023]


In [18]:
import numpy as np
import pickle
from scipy.spatial.distance import cdist
import os
import pandas as pd
output_dir = '/content/drive/My Drive/IR_Assignment2'
dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'
df = pd.read_csv(dataset_path)

# Assuming output_dir is already defined
results_path = os.path.join(output_dir, 'new_retrieval_results.pkl')

with open(results_path, 'rb') as f:
    retrieval_results = pickle.load(f)

# Extract individual components from the loaded retrieval results
similar_image_indices = retrieval_results['similar_image_indices']
image_similarities = retrieval_results['image_similarities']
similar_review_indices = retrieval_results['similar_review_indices']
review_similarities = retrieval_results['review_similarities']


def calculate_composite_scores(image_similarities, review_similarities):
    composite_scores = []
    for image_similarity, review_similarity in zip(image_similarities, review_similarities):
        # Calculate the average similarity score for each pair
        composite_score = (image_similarity + review_similarity) / 2
        composite_scores.append(composite_score)
    return composite_scores

composite_scores = calculate_composite_scores(image_similarities, review_similarities)

# Create a list of tuples (composite_score, image_index, review_index) and sort it
ranked_pairs = sorted(zip(composite_scores, similar_image_indices, similar_review_indices), reverse=True, key=lambda x: x[0])

# Display the ranked results
print("Ranked Combined Retrieval Results:")
for rank, (comp_score, img_idx, rev_idx) in enumerate(ranked_pairs, start=1):
    print(f"Rank: {rank}, Image Index: {img_idx}, Review Index: {rev_idx}, Composite Score: {comp_score:.4f}")


ranked_results_path = os.path.join(output_dir, 'ranked_combined_retrieval_results.pkl')
with open(ranked_results_path, 'wb') as f:
    pickle.dump(ranked_pairs, f)

print(f"Ranked combined retrieval results saved to: {ranked_results_path} \n")


def get_data_by_indices(df, image_indices, review_indices):
    # Extract image URLs and reviews by indices
    image_urls = df.loc[image_indices, 'Image'].tolist()
    reviews = df.loc[review_indices, 'Review Text'].tolist()
    return image_urls, reviews


image_urls, reviews = get_data_by_indices(df, similar_image_indices, similar_review_indices)



Ranked Combined Retrieval Results:
Rank: 1, Image Index: 0, Review Index: 0, Composite Score: 0.4976
Rank: 2, Image Index: 62, Review Index: 794, Composite Score: 0.4881
Rank: 3, Image Index: 193, Review Index: 750, Composite Score: 0.4724
Ranked combined retrieval results saved to: /content/drive/My Drive/IR_Assignment2/ranked_combined_retrieval_results.pkl 



In [19]:
output_dir = '/content/drive/My Drive/IR_Assignment2'
import pandas as pd
dataset_path = '/content/drive/My Drive/IR_Assignment2/A2_Data.csv'
df = pd.read_csv(dataset_path)

def calculate_composite_scores(image_similarities, review_similarities):
    composite_scores = []
    # Assuming image_similarities and review_similarities are aligned and of equal length
    for i in range(len(image_similarities)):
        comp_score = (image_similarities[i] + review_similarities[i]) / 2
        composite_scores.append((i, i, comp_score))  # Use i for both indices, or adjust as needed
    composite_scores.sort(key=lambda x: x[2], reverse=True)
    return composite_scores



## Calculate composite scores using only similarity scores
composite_scores = calculate_composite_scores(image_similarities, review_similarities)

## Display the combined retrieval results
print("USING IMAGE RETRIEVAL")
for i, (img_idx, rev_idx, comp_score) in enumerate(composite_scores, start=1):

     print(f"{i}) Image URL: {image_urls[img_idx]}")  # Example: Single URL or a list if applicable
     print(f"Review: {reviews[rev_idx]}")
     print(f"Cosine similarity of images - {image_similarities[img_idx]:.4f}")
     print(f"Cosine similarity of text - {review_similarities[rev_idx]:.4f}\n")


composite_image_score = sum(image_similarities) / len(image_similarities)
composite_text_score = sum(review_similarities) / len(review_similarities)
final_composite_score = (composite_image_score + composite_text_score) / 2

print("Composite similarity scores of images:", f"{composite_image_score:.4f}")
print("Composite similarity scores of text:", f"{composite_text_score:.4f}")
print("Final composite similarity score:", f"{final_composite_score:.4f}\n")


print("USING TEXT RETRIEVAL")
for i, (img_idx, rev_idx, comp_score) in enumerate(composite_scores, start=1):

     print(f"{i}) Image URL: {image_urls[img_idx]}")
     print(f"Review: {reviews[rev_idx]}")
     print(f"Cosine similarity of images - {image_similarities[img_idx]:.4f}")
     print(f"Cosine similarity of text - {review_similarities[rev_idx]:.4f}\n")

final_composite_image_score = sum(image_similarities) / len(image_similarities)
final_composite_text_score = sum(review_similarities) / len(review_similarities)
final_composite_score = (final_composite_image_score + final_composite_text_score) / 2

print("Composite similarity scores of images:", final_composite_image_score)
print("Composite similarity scores of text:", final_composite_text_score)
print("Final composite similarity score:", final_composite_score)

USING IMAGE RETRIEVAL
1) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81q5+IxFVUL._SY88.jpg']
Review: Loving these vintage springs on my vintage strat. They have a good tension and great stability. If you are floating your bridge and want the most out of your springs than these are the way to go.
Cosine similarity of images - 0.8524
Cosine similarity of text - 0.1428

2) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/71nSUnv7znL._SY88.jpg']
Review: Good
Cosine similarity of images - 0.8361
Cosine similarity of text - 0.1400

3) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81Eq6y34BYL._SY88.jpg']
Review: Great Quality, adjustable tension. Well made.
Cosine similarity of images - 0.8212
Cosine similarity of text - 0.1237

Composite similarity scores of images: 0.8366
Composite similarity scores of text: 0.1355
Final composite similarity score: 0.4860

USING TEXT RETRIEVAL
1) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81q